In [ ]:
#run train.py cifar10 cifar_resnet110 ./out --batch 512 --lr 0.01 --lr_step_size 32 --momentum 0.75 --base_sigma 0.00

In [ ]:
#run certify.py cifar10 ./out01_bias/checkpoint.pth.tar 0.12 ./out01_bias --skip 5000 --batch 512 --add_bias_layer True

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

import pickle as pkl
import itertools as itt

from pathlib import Path

In [ ]:
path = Path("./out01_bias/results.pkl")

In [ ]:
with open(path, "rb") as f:
    results = pkl.load(f)

In [ ]:
model_idx_list = [f"{i:02}" for i in range(1,11)]
suffix_list = ["", "_bias", "_bias_only", "_bias_only_sigma"]
iter_all = list(itt.product(model_idx_list, suffix_list))

In [ ]:
df = pd.DataFrame(columns=("suffix", "radius_all", "radius_cor", "model_idx"))
for model_idx, suffix in iter_all:
    path = Path(f"./out{model_idx}{suffix}/results.pkl")
    with open(path, "rb") as f:
        results = pkl.load(f)
    radius = np.array(results["certified_radius"])
    I_pos = radius > 0.0
    I_cor = np.array(results["correct"], dtype=bool)
    df = pd.concat(
        [
            df,
            pd.DataFrame([{
                "suffix": suffix,
                "model_idx": model_idx,
                "radius_all": radius[I_pos].mean(),
                "radius_cor": radius[I_cor].mean()}]
                        )
        ]
    )

In [ ]:
df.head()

In [ ]:
sns.boxplot(x="suffix", y="radius_all", data=df)

In [ ]:
sns.boxplot(x="suffix", y="radius_cor", data=df)